# Azure AI Agent service - Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import BingGroundingTool, MessageRole, ListSortOrder
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [2]:
load_dotenv("azure.env")

True

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 16-Jun-2025 14:31:50


## Grounding Bing Search

In [4]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

### Create connection to bing grounding service

In [5]:
# Shoud be: /subscriptions/<subscription_id>/resourceGroups/<resource_group_name>/providers/Microsoft.CognitiveServices/accounts/<ai_service_name>/projects/<project_name>/con#nections/<connection_name>
BING_CONN_ID = os.getenv("BING_CONN_ID")

In [6]:
bing = BingGroundingTool(connection_id=BING_CONN_ID)
bing

### Create agent with Bing Grounding tool

In [7]:
# Create the agent with the Bing grounding tool
instructions = "You are an AI Agent that can do some web search using Bing"

agent = project_client.create_agent(
    model="gpt-4o",
    temperature=0.7,
    name="Bing-grounding-agent-demo",
    instructions=instructions,
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Agent with Grounding Bing Search",
    )

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_6zJwGhgIjfV5eE2hKNLwZ3Dg


### Example 1

In [9]:
prompt = "What is the current price of the MICROSOFT stock in USD?"

In [10]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_RnfMvolLjdIgvXC4hhsDhJlR
💬 [2] Created message | ID: msg_pamV8rMqhBlEAAzpyMQKWjLi
🏃 [3] Created run | ID: run_CD6eu3S2SlJ4JBoP7PzBCtbo
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What is the current price of the MICROSOFT stock in USD?

💡 → As of June 5, 2025, the closing stock price for Microsoft (MSFT) is $467.68 USD【3:0†source】【3:1†source】.

🔗 URL Citation: [Microsoft Corporation (MSFT) Stock Historical Prices & Data - Yahoo Finance](https://finance.yahoo.com/quote/MSFT/history/)
🔗 URL Citation: [Microsoft (MSFT) Historical Stock Price Data - Stock Analysis](https://stockanalysis.com/stocks/msft/history/)



### Example 2

In [11]:
prompt = "What are the latest news about the Apple WWDC event?"

In [12]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_JOAVaw3MttbJri2UCaHta7YA
💬 [2] Created message | ID: msg_0bhGBm9Krzuk1Z1C3l0y0oY0
🏃 [3] Created run | ID: run_O3EMmAXlARoCR6yLOhkgJQ2i
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What are the latest news about the Apple WWDC event?

💡 → The latest updates from Apple's WWDC (Worldwide Developers Conference) 2025 include several major announcements:

1. **Software Updates**:
   - Apple introduced iOS 26, featuring a redesigned interface, new Live Translation capabilities, and a Liquid Glass UI.
   - VisionOS 26 for the Vision Pro headset focuses on "spatial experiences" with customizable widgets and an immersive media environment.
   - iPadOS 26 now includes a proper windowing system for multitasking, resembling a lightweight laptop experience.

2. **AI and Intelligence Features**:
   - Updates to Apple Intelligence, including AI-powered features across its ecosystem such as Sir

### Example 3

In [13]:
prompt = "What are the weather informations for Paris? Show the report and use emojis and be funny"

In [14]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_vG9a1LAK2VzNdPnoYMBhSc5f
💬 [2] Created message | ID: msg_iAgkDSnEpltgHZbcE4jL0T7x
🏃 [3] Created run | ID: run_mAOH4lmDTQ9BQvt99lGqdB65
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What are the weather informations for Paris? Show the report and use emojis and be funny

💡 → Bonjour from Paris! Here's your weather report for today, June 12, 2025, with a sprinkle of humor and emojis:

🌡️ **Temperature**: A toasty 84°F (29°C) in the day and a cozy 68°F (20°C) at night. It's hot enough to melt your croissant! 🥐🔥  
☀️ **Sky**: Clear and sunny—time to break out the berets and shades! 😎🧢  
💧 **Rain**: Not a drop in sight, so your Eiffel Tower selfies are safe. 📸✨  
🌬️ **Wind**: Gentle breezes caressing your face at around 6 mph, perfect for a stroll on the Seine. 🌬️🚶‍♂️  

Enjoy the city of lights with the perfect weather to match! 🌟🇫🇷【3:0†source】

🔗 URL Citation:

### Example 4

In [15]:
prompt = "What can you tell about Roland Garros 2025?"

In [16]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_5igZ6KIJg5jVrdxorqNuQyS4
💬 [2] Created message | ID: msg_2qFtcT5HGO6vCeT21xzWSi4N
🏃 [3] Created run | ID: run_gaowKXZDPLc8aSJQ4kTm3V4E
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What can you tell about Roland Garros 2025?

💡 → Roland Garros 2025, also known as the French Open, concluded on June 8, 2025. It featured top players from around the world on the iconic clay courts of Stade Roland Garros in Paris, France. Highlights include Carlos Alcaraz's victory in the men's final over Jannik Sinner, marking another milestone in his career. Coco Gauff claimed the women's singles title, securing her second Grand Slam win. Notable doubles performances included Alfie Hewett and Gordon Reid, who achieved their sixth consecutive men's wheelchair doubles title at Roland Garros【3:0†source】【3:1†source】【3:4†source】.

Let me know if you'd like more detailed insights in

## Example 5

In [17]:
prompt = "Who won at the Cannes festival in 2025?"

In [18]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_E3DbVcgt07RgLC3I2EuB4PQi
💬 [2] Created message | ID: msg_V5sFqIpwaVajWgDMqAqzbVSM
🏃 [3] Created run | ID: run_KdIj3zHxApzjNTjT3sP29VHE
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → Who won at the Cannes festival in 2025?

💡 → The Palme d'Or, the top prize at the 2025 Cannes Film Festival, was awarded to Iranian filmmaker Jafar Panahi for his movie *"It Was Just an Accident."* Other notable winners included:

- **Grand Prix**: *Sentimental Value* by Joachim Trier.
- **Joint Jury Prize**: *Sirât* by Oliver Laxe and *Sound of Falling* by Mascha Schilinski.
- **Best Director**: Kleber Mendonça Filho for *The Secret Agent*.
- **Best Actress**: Nadia Melliti for *The Little Sister*.
- **Best Actor**: Wagner Moura for *The Secret Agent*【3:0†source】【3:2†source】.

🔗 URL Citation: [The 78th Festival de Cannes winners’ list](https://www.festival-cannes.com/en/press/press-releases/the-78th-

### Post processing

In [21]:
print(
    f"Agent Details:\n"
    f"🆔 ID          : {agent.id}\n"
    f"📛 Name        : {agent.name}\n"
    f"🧠 Model       : {agent.model}\n"
    f"📜 Instructions:{agent.instructions}"
)

Agent Details:
🆔 ID          : asst_6zJwGhgIjfV5eE2hKNLwZ3Dg
📛 Name        : Bing-grounding-agent-demo
🧠 Model       : gpt-4o
📜 Instructions:You are an AI Agent that can do some web search using Bing


In [ ]:
#print("Deleting the agent we have created...")
#project_client.delete_agent(agent.id)
#print(f"Deleted agent, agent ID: {agent.id}")
#print("Done")